<a href="https://colab.research.google.com/github/bossham/Automation/blob/main/(20231222ver_)Crawler_for_Newsletter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🗞 ***CTis 뉴스레터 내용 수집 크롤러***

## ❗ 수정 (2023.12.20~22)

- '작성일' 열 삭제 및 '구분', '분류', '번호', 'url', '제목'으로 순서 변경
- 코드 간략화
- Nan 값으로만 이루어진 행 추가
- 수집한 데이터 제목 끝에 'NEW'가 붙으면 NEW 삭제
- 과거~최신순으로 데이터 정렬
- 서울시 새소식 크롤링 코드 업데이트 (sheet2로 저장)

---

In [ ]:
!pip install selenium
!pip install beautifulsoup4 requests pandas

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from datetime import datetime

## CTis 기사 순서 그대로 크롤링

In [ ]:
# def crawl_category(category_name, category_url, base_url, start_page, end_page, selector):
#     data_list = []

#     for page in range(start_page, end_page + 1):
#         url = category_url + str(page)

#         response = requests.get(url)
#         soup = BeautifulSoup(response.text, "html.parser")

#         articles = soup.select(selector)
#         for article in articles:
#             number = article.select_one("td:nth-child(1)").get_text(strip=True)
#             category = article.select_one("td:nth-child(2)").get_text(strip=True)
#             title = article.select_one("td:nth-child(3) a").get_text(strip=True)
#             date = article.select_one("td:nth-child(4)").get_text(strip=True)
#             link = article.select_one("td:nth-child(3) a")["href"]

#             link = urljoin(base_url, link)

#             # 추가 : 기사 제목 끝 'NEW' 삭제
#             title = title.rstrip('NEW').strip()

#             article_data = {
#                 "구분": category_name,
#                 "번호": number,
#                 "분류": category,
#                 "제목": title,
#                 "작성일": date,
#                 "링크": link,
#             }
#             data_list.append(article_data)

#     df = pd.DataFrame(data_list)

#     # 추가 : Nan행과 '구분'+Nan로 이뤄진 행 추가 -> category별 수집 데이터 분리
#     category_row = pd.DataFrame([category_name], columns=["구분"])
#     nan_row = pd.DataFrame([[float('nan')] * len(category_row.columns)], columns=category_row.columns)
#     df = pd.concat([nan_row, category_row, df], ignore_index=True)

#     # 추가 : '번호' 열 정수 변환
#     df['번호'] = pd.to_numeric(df['번호'], errors='coerce').astype('Int64')

#     return df

# def crawl_report_category(category_name, category_url, base_url, start_page, end_page):
#     data_list = []

#     for page in range(start_page, end_page + 1):
#         url = category_url + str(page)

#         response = requests.get(url)
#         soup = BeautifulSoup(response.text, "html.parser")

#         articles = soup.select(".news_in ul li")
#         for article in articles:
#             page_number = page
#             title = article.select_one("a.txt").get_text(strip=True)
#             article_url = article.select_one("a.txt")["href"]

#             article_url = urljoin(base_url, article_url)

#             article_data = {
#                 "구분": category_name,
#                 "번호": page_number,
#                 "제목": title,
#                 "링크": article_url,
#             }
#             data_list.append(article_data)

#     df = pd.DataFrame(data_list)

#     # 추가 : Nan행과 '구분'+Nan로 이뤄진 행 추가 -> category별 수집 데이터 분리
#     category_row = pd.DataFrame([category_name], columns=["구분"])
#     nan_row = pd.DataFrame([[float('nan')] * len(category_row.columns)], columns=category_row.columns)
#     df = pd.concat([nan_row, category_row, df], ignore_index=True)

#     # 추가 : '번호' 열 정수 변환
#     df['번호'] = pd.to_numeric(df['번호'], errors='coerce').astype('Int64')

#     df = pd.concat([pd.DataFrame([category_name], columns=["구분"]), df], ignore_index=True)
#     return df

# # 크롤링할 정보 정의
# categories_info = [
#     ("국제/국외소식", "https://www.ctis.re.kr/ko/selectBbsNttList.do?key=1688&bbsNo=313&searchCtgry=&pageUnit=10&searchCnd=all&searchKrwd=&searchBbsType=&searchKrwdType=&chgPage=&pageIndex=", "https://www.ctis.re.kr/ko/", 1, 10, ".jsgrid-row"),
#     ("국내동향", "https://www.ctis.re.kr/ko/selectBbsNttList.do?key=1692&bbsNo=312&searchCtgry=&pageUnit=10&searchCnd=all&searchKrwd=&searchBbsType=&searchKrwdType=&chgPage=&pageIndex=", "https://www.ctis.re.kr/ko/", 1, 10, ".jsgrid-row"),
#     ("기술동향", "https://www.ctis.re.kr/ko/selectBbsNttList.do?key=1696&bbsNo=345&searchCtgry=&pageUnit=10&searchCnd=all&searchKrwd=&searchBbsType=&searchKrwdType=&chgPage=&pageIndex=", "https://www.ctis.re.kr/ko/", 1, 5, ".jsgrid-row"),
#     ("국내 보고서", "https://www.ctis.re.kr/ko/selectBbsNttList.do?key=1694&bbsNo=314&searchCtgry=&pageUnit=8&searchCnd=all&searchKrwd=&searchBbsType=&pageIndex=", "https://www.ctis.re.kr/ko/", 1, 2, ".news_in ul li"),
#     ("국외 보고서", "https://www.ctis.re.kr/ko/selectBbsNttList.do?key=1690&bbsNo=348&searchCtgry=&pageUnit=8&searchCnd=all&searchKrwd=&searchBbsType=&pageIndex=", "https://www.ctis.re.kr/ko/", 1, 2, ".news_in ul li"),
#     ("국내 사업", "https://www.ctis.re.kr/ko/selectBbsNttList.do?key=1701&bbsNo=316&searchCtgry=&pageUnit=10&searchCnd=all&searchKrwd=&searchBbsType=&searchKrwdType=&chgPage=&pageIndex=", "https://www.ctis.re.kr/ko/", 1, 1, "table.jsgrid-table tbody tr"),
#     ("국제 사업", "https://www.ctis.re.kr/ko/selectBbsNttList.do?key=1700&bbsNo=346&searchCtgry=&pageUnit=10&searchCnd=all&searchKrwd=&searchBbsType=&searchKrwdType=&chgPage=&pageIndex=", "https://www.ctis.re.kr/ko/", 1, 1, "table.jsgrid-table tbody tr"),
# ]

# dataframes_list = []

# for category_info in categories_info:
#     category_name, category_url, base_url, start_page, end_page, selector = category_info
#     if "보고서" in category_name:
#         df = crawl_report_category(category_name, category_url, base_url, start_page, end_page)
#     else:
#         df = crawl_category(category_name, category_url, base_url, start_page, end_page, selector)
#     dataframes_list.append(df)

# combined_dataframe = pd.concat(dataframes_list, ignore_index=True)

## 편집회의 양식으로 역순으로 크롤링

In [ ]:
def crawl_category(category_name, category_url, base_url, start_page, end_page, selector):
    data_list = []

    for page in range(end_page, start_page - 1, -1):  # 역순으로 크롤링
        url = category_url + str(page)

        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")

        articles = soup.select(selector)
        for article in reversed(articles):  # 역순으로 데이터 추가
            number = article.select_one("td:nth-child(1)").get_text(strip=True)
            category = article.select_one("td:nth-child(2)").get_text(strip=True)
            title = article.select_one("td:nth-child(3) a").get_text(strip=True)
            date = article.select_one("td:nth-child(4)").get_text(strip=True)
            link = article.select_one("td:nth-child(3) a")["href"]

            link = urljoin(base_url, link)

            # 추가 : 기사 제목 끝 'NEW' 삭제
            title = title.rstrip('NEW').strip()

            # 역순으로 데이터 추가
            article_data = {
                "구분": category_name,
                "번호": number,
                "분류": category,
                "제목": title,
                "작성일": date,
                "링크": link,
            }
            data_list.append(article_data)

    df = pd.DataFrame(data_list)

    # 추가 : '번호' 열 정수 변환
    df['번호'] = pd.to_numeric(df['번호'], errors='coerce').astype('Int64')

    # 추가 : '구분' 값과 NaN 값으로 채워진 행을 데이터 윗 행에 배치
    nan_row = pd.DataFrame([[float('nan')] * len(df.columns)], columns=df.columns)
    category_row = pd.DataFrame([category_name], columns=["구분"])
    df = pd.concat([nan_row, category_row, df], ignore_index=True)

    return df

def crawl_report_category(category_name, category_url, base_url, start_page, end_page):
    data_list = []

    for page in range(end_page, start_page - 1, -1):  # 역순으로 크롤링
        url = category_url + str(page)

        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")

        articles = soup.select(".news_in ul li")
        for article in reversed(articles):  # 역순으로 데이터 추가
            page_number = page
            title = article.select_one("a.txt").get_text(strip=True)
            article_url = article.select_one("a.txt")["href"]

            article_url = urljoin(base_url, article_url)

            # 역순으로 데이터 추가
            article_data = {
                "구분": category_name,
                "번호": page_number,
                "제목": title,
                "링크": article_url,
            }
            data_list.append(article_data)

    df = pd.DataFrame(data_list)

    # 추가 : '번호' 열 정수 변환
    df['번호'] = pd.to_numeric(df['번호'], errors='coerce').astype('Int64')

    # 추가 : '구분' 값과 NaN 값으로 채워진 행을 데이터 윗 행에 배치
    nan_row = pd.DataFrame([[float('nan')] * len(df.columns)], columns=df.columns)
    category_row = pd.DataFrame([category_name], columns=["구분"])
    df = pd.concat([nan_row, category_row, df], ignore_index=True)

    return df

# 크롤링할 정보 정의
categories_info = [
    ("국제/국외소식", "https://www.ctis.re.kr/ko/selectBbsNttList.do?key=1688&bbsNo=313&searchCtgry=&pageUnit=10&searchCnd=all&searchKrwd=&searchBbsType=&searchKrwdType=&chgPage=&pageIndex=", "https://www.ctis.re.kr/ko/", 1, 10, ".jsgrid-row"),
    ("국내동향", "https://www.ctis.re.kr/ko/selectBbsNttList.do?key=1692&bbsNo=312&searchCtgry=&pageUnit=10&searchCnd=all&searchKrwd=&searchBbsType=&searchKrwdType=&chgPage=&pageIndex=", "https://www.ctis.re.kr/ko/", 1, 10, ".jsgrid-row"),
    ("기술동향", "https://www.ctis.re.kr/ko/selectBbsNttList.do?key=1696&bbsNo=345&searchCtgry=&pageUnit=10&searchCnd=all&searchKrwd=&searchBbsType=&searchKrwdType=&chgPage=&pageIndex=", "https://www.ctis.re.kr/ko/", 1, 5, ".jsgrid-row"),
    ("국내 보고서", "https://www.ctis.re.kr/ko/selectBbsNttList.do?key=1694&bbsNo=314&searchCtgry=&pageUnit=8&searchCnd=all&searchKrwd=&searchBbsType=&pageIndex=", "https://www.ctis.re.kr/ko/", 1, 2, ".news_in ul li"),
    ("국외 보고서", "https://www.ctis.re.kr/ko/selectBbsNttList.do?key=1690&bbsNo=348&searchCtgry=&pageUnit=8&searchCnd=all&searchKrwd=&searchBbsType=&pageIndex=", "https://www.ctis.re.kr/ko/", 1, 2, ".news_in ul li"),
    ("국내 사업", "https://www.ctis.re.kr/ko/selectBbsNttList.do?key=1701&bbsNo=316&searchCtgry=&pageUnit=10&searchCnd=all&searchKrwd=&searchBbsType=&searchKrwdType=&chgPage=&pageIndex=", "https://www.ctis.re.kr/ko/", 1, 1, "table.jsgrid-table tbody tr"),
    ("국제 사업", "https://www.ctis.re.kr/ko/selectBbsNttList.do?key=1700&bbsNo=346&searchCtgry=&pageUnit=10&searchCnd=all&searchKrwd=&searchBbsType=&searchKrwdType=&chgPage=&pageIndex=", "https://www.ctis.re.kr/ko/", 1, 1, "table.jsgrid-table tbody tr"),
]

dataframes_list = []

for category_info in categories_info:
    category_name, category_url, base_url, start_page, end_page, selector = category_info
    if "보고서" in category_name:
        df = crawl_report_category(category_name, category_url, base_url, start_page, end_page)
    else:
        df = crawl_category(category_name, category_url, base_url, start_page, end_page, selector)
    dataframes_list.append(df)

combined_dataframe = pd.concat(dataframes_list, ignore_index=True)

In [ ]:
combined_dataframe = combined_dataframe[['구분', '분류', '번호', '링크', '제목']]

In [ ]:
combined_dataframe

,구분,분류,번호,링크,제목
0,NaN,NaN,<NA>,NaN,NaN
1,국제/국외소식,NaN,<NA>,NaN,NaN
2,국제/국외소식,UNFCCC,10145,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,[UNFCCC] A Decade of REDD+: Notable Achievemen...
3,국제/국외소식,UNFCCC,10146,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,"[UNFCCC] Coal, Oil, and Gas are Fueling the Co..."
4,국제/국외소식,UNFCCC,10147,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,[UNFCCC] Closing Adaptation Knowledge Gaps in ...
...,...,...,...,...,...
311,국제 사업,CTCN TA,94,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,Call for Experts : Water-Energy-Food Nexus usi...
312,국제 사업,CTCN TA,95,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,Call for Experts: The Radio-Internet climate t...
313,국제 사업,CTCN TA,96,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,Call for Experts: developing real-time mapping...
314,국제 사업,CTCN TA,97,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,Call for Experts: Formulating a Grid Code and ...


In [ ]:
for category in combined_dataframe['구분'].unique():
    print(f"{category}\n")
    display(combined_dataframe[combined_dataframe['구분'] == category])
    print('\n')

nan



,구분,분류,번호,링크,제목




국제/국외소식



,구분,분류,번호,링크,제목
1,국제/국외소식,NaN,<NA>,NaN,NaN
2,국제/국외소식,UNFCCC,10145,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,[UNFCCC] A Decade of REDD+: Notable Achievemen...
3,국제/국외소식,UNFCCC,10146,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,"[UNFCCC] Coal, Oil, and Gas are Fueling the Co..."
4,국제/국외소식,UNFCCC,10147,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,[UNFCCC] Closing Adaptation Knowledge Gaps in ...
5,국제/국외소식,기타,10148,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,[UN] Explainer: What’s COP28 and why is it imp...
...,...,...,...,...,...
97,국제/국외소식,UNFCCC,10240,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,"[UNFCCC] ""We didn’t turn the page on the fossi..."
98,국제/국외소식,UNFCCC,10241,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,[UNFCCC] Empowering Nations with Tools for Rob...
99,국제/국외소식,UNFCCC,10242,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,[UNFCCC] Empowering Nations with Tools for Rob...
100,국제/국외소식,UNFCCC,10243,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,[UNFCCC] Building Momentum for Climate Action ...




국내동향



...

,구분,분류,번호,링크,제목
275,국외 보고서,NaN,<NA>,NaN,NaN
276,국외 보고서,NaN,2,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,[CTCN] SRIF
277,국외 보고서,NaN,2,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,[CTCN] SRIF
278,국외 보고서,NaN,2,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,[CTCN] SRIF
279,국외 보고서,NaN,2,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,[CTCN] Request for Fortalecimiento de las capa...
280,국외 보고서,NaN,2,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,[CTCN] SRIF for Soluciones basadas en la natur...
281,국외 보고서,NaN,2,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,[CTCN] Response Plan signed
282,국외 보고서,NaN,2,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,[CTCN] FINAL Response Plan Ecuador
283,국외 보고서,NaN,2,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,[CTCN] Implementation Plan
284,국외 보고서,NaN,1,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,[CTCN] test




국내 사업



,구분,분류,번호,링크,제목
293,국내 사업,NaN,<NA>,NaN,NaN
294,국내 사업,EDCF,552,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,[긴급] EDCF 기후변화영향 대응체계』 시범적용(2023년 3차)
295,국내 사업,기타,553,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,「한국형 녹색분류체계 4개 환경목표 경제활동 도출 연구」 입찰 공고
296,국내 사업,기타,554,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,한국형 녹색분류체계 4개 환경목표 경제활동 도출 연구」 입찰 공고(새로운 공고)
297,국내 사업,기타,555,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,기후위기 대응을 위한 재해 대응체계 개선방안 연구
298,국내 사업,기타,556,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,2023년도 기후기술국제협력촉진사업(CTCN 기술지원) 신규과제 3차 공고
299,국내 사업,기타,557,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,신재생에너지 통합관제 현황 및 제도개선 연구
300,국내 사업,기타,558,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,기후위기 대응 수산자원 변화 예측 기반 수산자원 관리연구 로드맵 수립 연구
301,국내 사업,EDCF,559,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,[긴급]『EDCF 기후변화영향 대응체계』 시범적용 (2023년 5차)
302,국내 사업,기타,560,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,탄소중립 이행 프로그램 사업(RND) 사전기획 연구




국제 사업



,구분,분류,번호,링크,제목
305,국제 사업,NaN,<NA>,NaN,NaN
306,국제 사업,CTCN TA,89,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,Seeking experts: CTCN project in South Africa
307,국제 사업,CTCN TA,90,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,CTCN Vacancy Announcement: Senior Programme Ma...
308,국제 사업,기타,91,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,[니카라과] 입찰정보
309,국제 사업,MDB,92,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,[사업 ADB] India: Proposed Project Readiness Fin...
310,국제 사업,MDB,93,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,[사업 ADB] Georgia: Georgian Green Bond 2 Project
311,국제 사업,CTCN TA,94,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,Call for Experts : Water-Energy-Food Nexus usi...
312,국제 사업,CTCN TA,95,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,Call for Experts: The Radio-Internet climate t...
313,국제 사업,CTCN TA,96,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,Call for Experts: developing real-time mapping...
314,국제 사업,CTCN TA,97,https://www.ctis.re.kr/ko/selectBbsNttView.do?...,Call for Experts: Formulating a Grid Code and ...


## 서울시 > 환경 > 새소식 역순 크롤링

https://news.seoul.go.kr/env/news-all?category&keyword&category_1=0&category_2=0

In [ ]:
# 빈 리스트 생성
titles = []
links = []

# 페이지 수 설정
start_page = 1
end_page = 3

# 페이지 순환
for page in range(end_page, start_page - 1, -1):
    # URL 생성
    url = f"https://news.seoul.go.kr/env/news-all/page/{page}?category&keyword&category_1=0&category_2=0"

    # 페이지 요청 및 응답 받기
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 포스트 목록 선택
    posts = soup.select('.post-lst > div')

    # 각 포스트에서 제목과 링크 추출
    for post in reversed(posts):
        title = post.select_one('.tit a').text.strip()
        link = post.select_one('.tit a')['href']

        titles.append(title)
        links.append(link)

# 데이터프레임 생성
seoul = pd.DataFrame({'제목': titles, '링크': links})

In [ ]:
seoul = seoul[['링크', '제목']]
seoul

,링크,제목
0,https://news.seoul.go.kr/env/archives/555895,중랑물재생센터 연료전지 사업 주민설명회 개최 안내
1,https://news.seoul.go.kr/env/archives/555930,문화비축기지 T5 기획전 <서용선 프로젝트 : 암태도>
2,https://news.seoul.go.kr/env/archives/555986,2023년 소형 빗물이용시설 설치비 추가지원 알림
3,https://news.seoul.go.kr/env/archives/556017,[서부공원여가센터] 월드컵공원 <2023 새활용 크리스마스 '12월...
4,https://news.seoul.go.kr/env/archives/556054,제4회 식재설계공모전 수상정원을 서울식물원에서 만나보세요
5,https://news.seoul.go.kr/env/archives/556071,서울시 산불예방 공익광고
6,https://news.seoul.go.kr/env/archives/556138,서울시 물재생센터 하수처리 현황(2020~2022년)
7,https://news.seoul.go.kr/env/archives/556181,`5차 미세먼지 계절관리제` 시행…4대 분야 16개 미세먼지 저감 대책
8,https://news.seoul.go.kr/env/archives/556225,2024년 문화비축기지 장소사용(대관) 정기 모집 공고
9,https://news.seoul.go.kr/env/archives/556285,"미세먼지 계절관리제, 맑은 서울을 향한 행복한 동행!"


## 엑셀로 추출

In [ ]:
!pip install xlsxwriter

In [ ]:
current_time = datetime.now().strftime('%Y-%m-%d_%H-%M')
excel_filename = f'{current_time}.xlsx'
excel_writer = pd.ExcelWriter(excel_filename, engine='xlsxwriter')

combined_dataframe.to_excel(excel_writer, sheet_name='CTis_Crawling', index=False)
seoul.to_excel(excel_writer, sheet_name='Seoul_Crawling', index=False)

excel_writer.save()

<ipython-input-23-49b2fd9529ed>:8: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  excel_writer.save()


In [ ]:
# current_datetime = datetime.now().strftime('%Y%m%d')
# file_name = f'newsletter_{current_datetime}.xlsx'
# combined_dataframe.to_excel(file_name, index=False)
# print(f"엑셀 파일이 저장되었습니다: {file_name}")